Aluna: Thais Luca Marques de Almeida

DRE: 122024801

# Código para as Questões 7, 8, 9 e 10

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import sys

Funções Auxiliares

In [2]:
def get_line_plot(x,func):
    '''
        Obtém os coeficientes angular e linear da reta da função
        no formato y(x) = mx + b

        Args:
            x: pontos no eixo x
            func: vetor de pesos [w0,w1,w2] correspondente
        Result:
            valores correspondentes aos parâmetros func 
            e x passados como parâmetro
    '''

    # Teste para evitar divisão por zero
    if np.all(func==0):
        return np.zeros(len(x))

    m = -func[0]/func[1] # coeficiente angular
    b = -func[2]/func[1] # coeficiente linear
    
    # Gera a linha dados os pontos do eixo x
    return x * m + b

def plot(X,y_true,h_func):
    '''
        Função que "plota" os dados de treinamento junto à hipótese

        Args:
            X: valores dos dados de treinamento [-1,1] x [-1,1]
            Y: labels dos dados de treinamento 
            h_func: função hipótese do perceptron

    '''

    plt.figure()
    
    # Filtra e plota os exemplos negativos
    x_negs = [X[i][0] for i in range(len(X)) if y_true[i] < 0] 
    y_negs = [X[i][1] for i in range(len(X)) if y_true[i] < 0]
    plt.scatter(x_negs,y_negs,color='red')
    
    # Filtra e plota os exemplos positivos
    x_pos = [X[i][0] for i in range(len(X)) if y_true[i] > 0]
    y_pos = [X[i][1] for i in range(len(X)) if y_true[i] > 0]
    plt.scatter(x_pos,y_pos,color='green')
    
    # Plota a hipótese
    xplt = np.linspace(-1, 1, 100)
    yplt2 = get_line_plot(xplt, h_func)
    plt.plot(xplt,yplt2,color='black',label='$g(x)$')

    plt.legend()
    plt.grid(True)
    
    plt.show()

def create_target_function():
    '''
        Gera a função target que determina a classificação dos dados de treinamento

        Result:
            array com três posições w0 = 0, w1 = m, w2 = b, 
            onde m e b são os coeficientes angular e linear, 
            respectivamente
    '''

    #Pego dois pontos aleatórios no espaço [-1,1] x [-1,1]
    p0 = random.uniform(-1,1), random.uniform(-1,1)
    p1 = random.uniform(-1,1), random.uniform(-1,1)

    #Calcula o coeficiente angular 
    m = (p1[1]-p0[1])/(p1[0]-p0[0])
    
    # Calcula o coeficiente linear
    b = p0[1] - (p0[0] * m)
    
    # Gero a função target 
    # w0 = m, w1 = -1, w2= b
    # y = m*x + b -> m*x -y + b = 0
    return np.array([m,-1,b])

def generate_training_data(N,f_target):
    '''
        Gera um conjunto de dados linearmente separável pela função target 
        (desconhecida pelo perceptron)

        Args:
            N: número de exemplos a serem gerados
            f_target: função target que separa esses dados
        Result:
            X_sample: vetor contendo os pontos (x,y) dos dados de treinamento e;
            y_sample: vetor contendo as labels dos dados de treinamento
    '''
    y_sample = []

    # Gerando a amostra de N pontos no espaço [-1,1] x [-1,1]
    X_sample = np.random.uniform(low=-1, high=1, size=(N,2))
    X_sample = [np.concatenate((X_sample[i],np.array([1.]))) for i in range(N)]
    for i in range(N):
        y_sample.append(1 if np.dot(f_target,X_sample[i]) > 0 else -1) # Classificação do ponto gerado segundo a função target passada como parâmetro
    
    return X_sample, np.array(y_sample)

***Classe Perceptron***

In [3]:
class Perceptron:
    '''
        Classe que implementa o Perceptron e o PLA
    '''

    def __init__(self,show_plot=False):
        self.g = np.zeros(3)
        self.show_plot = show_plot

    def __sign(self,func,pt):
        '''
            Define a classificação do ponto de acordo com o produto interno 
            entre o ponto e a função
            
            Args:
                func: função que separa os dados em +1 ou -1
                pt: ponto a ser classificado
            Result:
                valor inteiro correspondente ao rótulo do dado segundo func
        '''
        return 1 if np.dot(func,pt) > 0 else -1

    def __update_weights(self,X,y):
        '''
            Atualiza o vetor de pesos da função hipótese g
            O vetor de pesos é atualizado de acordo com um dos pontos 
            classificados incorretamente
            O ponto é escolhido de forma aleatória dentro do conjunto

            Args:
                X: dados de treinamento classificados de forma incorreta
                y: classificação dada por g (incorreta)
            Result:
                os pesos da reta g atualizados de acordo com um dos pontos
                classificados de forma incorreta escolhido de forma aleatória
        '''

        #Escolhe um ponto aleatoriamente
        i = random.randint(0,len(X)-1)

        # weight vector
        w = self.g
        
        #Atualiza os pesos
        # w(t+1) = w(t) + y(t)*x(t)
        # Nesse caso, vamos usar w(t+1) = w(t) - y(t)*x(t) 
        # porque consideramos m*x - y + b = 0
        w = w - y[i]*X[i]
        
        return w

    def __get_miss_classified_examples(self,X,y_pred,y_true):
        '''
            Compara os rótulos de cada exemplo y_pred com y_true para retornar 
            quais pontos não foram classificados corretamente por g

            Args:
                X: pontos (x,y) do conjunto de treinamento
                y_pred: rótulos previstos por g (hipótese)
                y_true: valores reais do rótulos de acordo com a função target 
                desconhecida
            Returns:
                as coordenadas dos pontos classificados incorretamente 
                e os rótulos divergentes dos rótulos reais
        '''
        X_mislabeled,y_mislabeled = [],[]
        for i in range(len(y_true)):
            if y_pred[i] != y_true[i]:
                X_mislabeled.append(X[i])
                y_mislabeled.append(y_pred[i])
        return X_mislabeled,y_mislabeled

    def predict(self, X, function):
        '''
            Realiza a predição de todos os pontos do conjunto de treinamento 
            de acordo com a função passada como parâmetro

            Args:
                X: pontos (x,y) do conjunto de treinamento
                function: função que irá classificar os pontos
            Return:
                lista de rótulos de classificação de acordo com a 
                função function
        '''
        return [self.__sign(function,x) for x in X]

    def train(self,X,y):
        '''
            Perceptron Learning Algorithm

            Começando com g contendo apenas valores nulos, 
            a reta é atualizada de acordo com os exemplos que 
            não foram classificados corretamente.
            O algoritmo termina quando não há mais pontos classificados 
            incorretamente.
            
            Args:
                X: pontos (x,y) do conjunto de treinamento
                y: rótulos dos pontos passados como parâmetro
            Return:
                número de iterações necessárias para conversão do modelo
        '''

        # Inicializo o número de iterações como zero
        n_iteractions = 0

        while True:

            # Inicializo os arrays de predição 
            # e exemplos que não foram classificados corretamente
            X_miss_classified, y_miss_classified, predicted = [], [], []

            # Predição dos exemplos pela função hipótese
            predicted = self.predict(X,self.g)
        
            # Coleto todos os pontos que não foram classificados corretamente
            X_miss_classified,y_miss_classified = self.__get_miss_classified_examples(X,predicted,y)
        
            #print(f'Número de pontos classificados de forma errada: {len(X_miss_classified)}')

            # Desenho os pontos e as funções na tela (opcional)
            if self.show_plot:
                plot(X,y,self.g)

            # Se todos os pontos foram classificados corretamente, 
            # encerra o programa
            if len(X_miss_classified) == 0:
                break

            # Uso os pontos que não foram classificados corretamente 
            # para ajustar os pesos de g
            self.g = self.__update_weights(X_miss_classified,y_miss_classified)
            
            # Cada vez que atualizo os pesos, conto uma iteração
            n_iteractions += 1
            
        return n_iteractions

# Experimentos

***Questão 7***

Média de iterações necessárias para conversão do PLA usando 10 pontos.

In [4]:
# Quantidade de pontos e interações
N = 10
total_iteractions = 0

# True para mostrar o gráfico a cada iteração; False cc
show_plot = False

for i in range(1000):

    # Inicializa a função objetivo
    f_target = create_target_function()

    # Gera o conjunto de treinamento
    X,y = generate_training_data(N,f_target)

    # Inicializa a classe perceptron
    model = Perceptron(show_plot=show_plot)

    # Treinamento do modelo
    n_iteractions = model.train(X,y)

    # Armazena o número de iterações da rodada atual
    total_iteractions += n_iteractions

print(total_iteractions/1000)

10.437


Resposta letra B. 

***Questão 8***

Média de probabilidade de f(x) != g(x) para 10 pontos.

In [5]:
# Quantidade de pontos e número de vezes que f foi diferente de g
N = 10
n_divergencies = 0

# True para mostrar o gráfico a cada iteração; False cc
show_plot = False

for i in range(1000):

    # Inicializa a função objetivo 
    f_target = create_target_function()

    # Gera o conjunto de treinamento para treinar o Perceptron
    X,y = generate_training_data(N,f_target)

    # Inicializa a classe perceptron
    model = Perceptron(show_plot=show_plot)

    # Treinamento do modelo
    n_iteractions = model.train(X,y)
    
    # Gero novos pontos para testar a divergência entre a função target e a hipótese aprendida
    new_X, new_y = generate_training_data(N,f_target)
    
    # Uso o modelo pra fazer a predição dos novos dados
    preds = []
    preds = model.predict(new_X,model.g)
    for y_true,y_pred in zip(new_y,preds):
        if y_true != y_pred:
            n_divergencies += 1/N

print(n_divergencies/1000)

0.10979999999999804


Resposta letra C. 

***Questão 9***

Média de iterações necessárias para conversão do PLA usando 100 pontos.

In [6]:
# Quantidade de pontos e interações
N = 100
total_iteractions = 0

# True para mostrar o gráfico a cada iteração; False cc
show_plot = False

for i in range(1000):

    # Inicializa a função objetivo
    f_target = create_target_function()

    # Gera o conjunto de treinamento
    X,y = generate_training_data(N,f_target)

    # Inicializa a classe perceptron
    model = Perceptron(show_plot=show_plot)

    # Treinamento do modelo
    n_iteractions = model.train(X,y)
    
    # Armazena o número de iterações da rodada
    total_iteractions += n_iteractions

print(total_iteractions/1000)

92.998


Resposta letra B.

***Questão 10***

Média de probabilidade de f(x) != g(x) para 100 pontos.

In [7]:
# Quantidade de pontos e número de vezes que f foi diferente de g
N = 100
n_divergencies = 0

# True para mostrar o gráfico a cada iteração; False cc
show_plot = False

for i in range(1000):

    # Inicializa a função objetivo
    f_target = create_target_function()

    # Gera o conjunto de treinamento
    X,y = generate_training_data(N,f_target)

    # Inicializa a classe perceptron
    model = Perceptron(show_plot=show_plot)

    # Treinamento do modelo
    n_iteractions = model.train(X,y)
    
    # Gero novos pontos para testar a divergência entre a função target e a hipótese aprendida
    new_X, new_y = generate_training_data(N,f_target)
    
    # Uso o modelo pra fazer a predição dos novos dados
    preds = []
    preds = model.predict(new_X,model.g)
    for y_true,y_pred in zip(new_y,preds):
        if y_true != y_pred:
            n_divergencies += 1/N

print(n_divergencies/1000)

0.012619999999999775


Resposta letra B.

## Interpretação dos Resultados

Como os dados são linearmente separáveis, o algoritmo consegue convergir a partir da quantidade de pontos usados no treinamento. Analisando o número de iterações necessárias para convergência, a média se dá pelo número de pontos presentes no conjunto de treinamento. Ou seja, a quantidade de vezes que a reta precisa se ajustar. Quando aumentamos consideravelmente o número de pontos do conjunto, esse valor também aumenta, uma vez que o algoritmo tem mais pontos para ajustar os pesos da função hipótese. Já quando observamos a probabilidade de g divergir de f, também podemos observar que quando aumentamos o número de pontos do conjunto, podemos encontrar retas melhores. Temos uma maior precisão na classificação dos pontos, o que diminui a probabilidade de g divergir de f. Quando utilizados poucos pontos (N=10), temos uma maior probabilidade de g divergir de f porque podemos encontrar retas que não descrevem tão bem a função. Quanto maior o número de pontos, mais informação temos para descrever a função. Consequentemente, maior o número de iterações.